In [1]:
import pandas as pd
from ipywidgets import Layout, Box, widgets
from astropy.io.votable import parse
from astroquery.simbad import Simbad
import astropy.units as u
from astropy.table import QTable
from ipyaladin import Aladin
from ipywidgets import Layout, Box, widgets
from astropy.samp import SAMPIntegratedClient
from urllib.parse import urljoin
import os
from astropy.io.votable.tree import VOTableFile
from astropy.table import Table

In [2]:
def votable_to_pandas(votable_file):
    '''
    Converts votable to pandas dataframe.
    '''
    votable = parse(votable_file)
    table = votable.get_first_table().to_table(use_names_over_ids=True)
    return table.to_pandas()

In [3]:
data = votable_to_pandas("../../data/cscresults.vot")

In [4]:
cl_0 = pd.read_csv('./class_data/cl0.csv', index_col=0)
cl_1 = pd.read_csv('./class_data/cl1.csv', index_col=0)
cl_2 = pd.read_csv('./class_data/cl2.csv', index_col=0)
cl_3 = pd.read_csv('./class_data/cl3.csv', index_col=0)
cl_4 = pd.read_csv('./class_data/cl4.csv', index_col=0)
cl_5 = pd.read_csv('./class_data/cl5.csv', index_col=0)

In [5]:
cl_all = pd.concat([cl_0, cl_1, cl_2, cl_4, cl_5])
cl_all_with_coords = cl_all.merge(data[['name','obsid', 'ra', 'dec']], how='left', on=['name', 'obsid'])

In [6]:
cl_all_with_coords.columns

Index(['name', 'obsid', 'hard_hm', 'hard_hs', 'hard_ms', 'powlaw_gamma',
       'bb_kt', 'var_prob_b', 'var_ratio_b', 'var_prob_h', 'var_ratio_h',
       'var_prob_s', 'var_ratio_s', 'var_newq_b', 'main_type', 'Orion_V*',
       'PartofG', 'QSO', 'TTau*', 'YSO', 'AGN', 'HMXB', 'Candidate_XB*',
       'Seyfert_1', 'Seyfert_2', 'ra', 'dec'],
      dtype='object')

In [7]:
def confident_detections(source_type, confidence=None):
    if confidence is not None:
        q_res = cl_all_with_coords[cl_all_with_coords[source_type] > confidence]
        print(f'{len(q_res)} source detections found of class {source_type} with a confidence of {confidence}...')
    else:
        q_res = cl_all_with_coords[cl_all_with_coords['main_type'] == source_type]
        print(f'{len(q_res)} source detections found of class {source_type}...')
    q_res_astro = Table.from_pandas(q_res)
    q_res_vot = VOTableFile.from_table(q_res_astro)
    #q_res_vot.to_xml('confident/{}.xml'.format(source_type.replace('*','_')))
    #q_res.to_csv(f'confident/{source_type}.csv')
    return q_res_astro

In [8]:
# For sending data to Aladin through SAMP
client = SAMPIntegratedClient()
client.connect()

SAMPHubError: Unable to find a running SAMP Hub.

In [ ]:
# Plot all
available_types = ['Orion_V*',
       'PartofG', 'QSO', 'TTau*', 'YSO', 'AGN', 'HMXB', 'Candidate_XB*',
       'Seyfert_1', 'Seyfert_2']

for at in available_types:
    source_type = at
    #q_res_astro = confident_detections(source_type)

    params = {}
    params["name"] = source_type
    params["url"] = 'file:' + urljoin('file:', os.path.abspath('confident/{}.xml'.format(source_type.replace('*','_'))))

    message = {}
    message["samp.mtype"] = "table.load.votable"
    message["samp.params"] = params

    client.notify_all(message)

## Aladin

#### YSO

In [35]:
a = Aladin(target='M42', fov=10, coo_frame="galactic", survey='P/allWISE/color')

In [36]:
a

Aladin(coo_frame='galactic', fov=10.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log'…

In [37]:
source_type = 'YSO'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

2047 source detections found of class YSO...


In [38]:
source_type = 'TTau*'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

450 source detections found of class TTau*...


In [39]:
source_type = 'Orion_V*'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

1217 source detections found of class Orion_V*...


#### XB

In [87]:
a = Aladin(target='M33', fov=10, coo_frame="galactic", survey='P/allWISE/color')
a

Aladin(coo_frame='galactic', fov=10.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log'…

In [88]:
source_type = 'HMXB'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

3656 source detections found of class HMXB...


In [89]:
source_type = 'Candidate_XB*'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

385 source detections found of class Candidate_XB*...


#### AGN

In [90]:
a = Aladin(target='M33', fov=10, coo_frame="galactic", survey='P/allWISE/color')
a

Aladin(coo_frame='galactic', fov=10.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log'…

In [91]:
source_type = 'AGN'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

1400 source detections found of class AGN...


In [92]:
source_type = 'QSO'
q_res_astro = confident_detections(source_type)
a.add_table(q_res_astro)

2766 source detections found of class QSO...
